In [40]:
import sys  
sys.path.insert(0, '..')

from datetime import date, timedelta

from config import Config

import pandas as pd
from sqlalchemy import create_engine

db = create_engine(Config.SQLALCHEMY_DATABASE_URI)

In [41]:
ockovani = pd.read_sql_query(
    """
    select vekova_skupina, vakcina, poradi_davky, sum(pocet) pocet_ockovani
    from ockovani_lide
    group by vekova_skupina, vakcina, poradi_davky
    """,
    db
)
# ockovani

In [42]:
registrace = pd.read_sql_query(
    """
    select vekova_skupina, sum(pocet) pocet_fronta
    from ockovani_registrace
    where rezervace = false and import_id = 31
    group by vekova_skupina
    """,
    db
)
# registrace

In [43]:
populace = pd.read_sql_query(
    """
    select vekova_skupina, sum(pocet) pocet_vek
    from populace p 
    join populace_kategorie k on (k.min_vek <= vek and k.max_vek >= vek)
    where orp_kod = 'CZ0'
    group by vekova_skupina
    """,
    db
)
# populace

In [44]:
ockovani['vekova_skupina'] = ockovani['vekova_skupina'].replace(['nezařazeno'], 'neuvedeno')
ockovani['pocet_ockovani_castecne'] = ockovani['pocet_ockovani'].where(ockovani['poradi_davky'] == 1).fillna(0).astype('int')
ockovani['pocet_ockovani_plne'] = ockovani['pocet_ockovani'].where((ockovani['poradi_davky'] == 2) | (ockovani['vakcina'].isin([]))).fillna(0).astype('int')
ockovani_grp = ockovani.groupby(['vekova_skupina']).sum().drop('poradi_davky', axis=1).reset_index()

merged = pd.merge(ockovani_grp, registrace, how="left")
merged = pd.merge(merged, populace, how="left")

merged['podil_ockovani_castecne'] = merged['pocet_ockovani_castecne'] / merged['pocet_vek']
merged['podil_ockovani_plne'] = merged['pocet_ockovani_plne'] / merged['pocet_vek']
merged

,vekova_skupina,pocet_ockovani,pocet_ockovani_castecne,pocet_ockovani_plne,pocet_fronta,pocet_vek,podil_ockovani_castecne
0,0-17,506,329,0.000089,89,1999465.0,0.000165
1,18-24,21461,14565,0.010265,2721,671799.0,0.021681
2,25-29,33559,21823,0.018003,5172,651892.0,0.033476
3,30-34,37366,24476,0.017850,6051,722127.0,0.033894
4,35-39,51274,34027,0.022604,8937,762998.0,0.044596
5,40-44,76764,51848,0.026982,12775,923446.0,0.056146
6,45-49,88064,59363,0.034197,12371,839278.0,0.070731
7,50-54,76807,52951,0.034764,8398,686236.0,0.077162
8,55-59,83028,59922,0.035357,6995,653501.0,0.091694
9,60-64,64728,46514,0.027954,2960,651567.0,0.071388
